<h2>Mask nuclei labels from full_image predictions</h2>

This notebook is useful if you have generated the nuclei predictions for the full image but did not draw the ROIs in advance.

In order to use this notebook you will need to run 002_BP_Predict_nuclei_labels.ipynb to generate the full_image nuclei labels

In [ ]:
from pathlib import Path
import tifffile
import os
import numpy as np
from tqdm import tqdm
from utils_stardist import list_images

In [52]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_big_data")

# Construct ROI path from directory_path above
input_roi_path = directory_path / "ROIs"

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "3D"

# Nuclear segmentation model type ("Stardist")
# Choose your Stardist fine-tuned model (model_name) from stardist_models folder
model_name = "3D_seg_stardist_v1.8"

# TODO: Set remove_edge_labels to True if you want to discard labels touching the edge of the ROI
remove_edge_labels = False

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['..\\raw_data\\test_big_data\\1482_naive_TR1.nd2',
 '..\\raw_data\\test_big_data\\1482_naive_TR2.nd2',
 '..\\raw_data\\test_big_data\\2106_conditioned_TR1.nd2',
 '..\\raw_data\\test_big_data\\2106_conditioned_TR2.nd2',
 '..\\raw_data\\test_big_data\\2107_conditioned_TR1.nd2',
 '..\\raw_data\\test_big_data\\2107_conditioned_TR2.nd2',
 '..\\raw_data\\test_big_data\\2108_conditioned_TR1.nd2']

In [53]:
# List of subfolder names
try:
    roi_names = [folder.name for folder in input_roi_path.iterdir() if folder.is_dir()]
    print(f"The following regions of interest will be applied over the full image nuclei: {roi_names}")
except FileNotFoundError:
    print("You have not defined any regions of interest. Please define them in '001_BP_Draw_ROIs.ipynb'")

The following regions of interest will be applied over the full image nuclei: ['CA', 'DG']


In [ ]:
for image in tqdm(images):

    try:
        # Generate Path object and extract filename
        file_path = Path(image)
        filename = file_path.stem
        # Read full_iamge nuclei_preds label image
        nuclei_labels = tifffile.imread(directory_path / "nuclei_preds" / segmentation_type / model_name / "full_image" / f"{filename}.tiff")
        print(f"Found full_image nuclei labels for: {filename}")

    except FileNotFoundError:
        print ("First you need to generate the full_image nuclei predictions in '002_BP_Predict_nuclei_labels.ipynb'")

    for roi_name in roi_names:

        if roi_name == "full_image":
            continue  # Skip the "full_image" roi_name

        # Check if the ROI prediction has already been generated
        try:
            tifffile.imread(directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name / f"{filename}.tiff")
            print(f"\nWARNING: Nuclei predictions already found for: {filename} ROI: {roi_name}")
            print("Make sure nuclei labels were generated using the same settings.")
            continue  # Skip to the next roi_name if the prediction exists
        except FileNotFoundError:

            # Construct path to read ROI
            roi_path = directory_path / "ROIs" / roi_name / f"{filename}.tiff"

            try:
                # Read the .tiff files containing the user-defined ROIs
                roi = tifffile.imread(roi_path)
            except FileNotFoundError:
                print(f"Missing ROI '{roi_name}'  for: '{filename}'")
                continue # Skip to the next roi_name if the ROI is not defined

            print(f"Generating masked nuclei labels for: {filename} ROI: {roi_name}")

            # We will create a mask where roi is greater than or equal to 1
            mask = (roi >= 1).astype(np.uint8)

            # 3D segmentation logic, extend 2D mask across the entire stack volume
            if segmentation_type == "3D":
                # Extract the number of z-slices to extend the mask
                slice_nr = nuclei_labels.shape[0]

                # Extend the mask across the entire volume
                mask = np.tile(mask, (slice_nr, 1, 1))

                # Apply the mask to nuclei_labels, setting all other pixels to 0
                masked_nuclei_labels = np.where(mask, nuclei_labels, 0)
            else:
                # Apply the mask to nuclei_labels, setting all other pixels to 0
                masked_nuclei_labels = np.where(mask, nuclei_labels, 0)

            # Save nuclei labels as .tiff files to reuse them later
            # Create nuclei_predictions directory if it does not exist
            try:
                os.makedirs(directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name)
            except FileExistsError:
                pass

            # Construct path to store
            nuclei_preds_path = directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name / f"{filename}.tiff"

            # Save nuclei labels as .tiff
            tifffile.imwrite(nuclei_preds_path, masked_nuclei_labels)      